In [1]:
import DualwormFunctions as dw
import Observables as obs
%load_ext line_profiler
%load_ext memory_profiler
import RunBasis
import argparse
import StartStates as strst
import numpy as np

In [2]:
nst = 2;
nsm = 32;
nips = 2;
nrps = 4;
nb = 32;
measperiod = 2;
nt = 8
nh = 8

In [3]:
def genArgs():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-f', type = str)

    parser.add_argument('--L', type = int, default = 12, help = 'Lattice side size')

    # COUPLINGS
    parser.add_argument('--J1', type = float, default = 1,
                        help = 'NN coupling') # nearest-neighbour coupling
    parser.add_argument('--J2', type = float, default = 0,
                        help = '2nd NN coupling') # 2nd NN coupling
    parser.add_argument('--J3', type = float, default = 0,
                        help = '3rd NN coupling') # 3rd NN coupling
    parser.add_argument('--J4', type = float, default = 0.0,
                        help = '4th NN coupling')
    parser.add_argument('--h', type = float, default = 1.0,
                        help = 'Magnetic field')    
    #NUMBER OF STEPS AND ITERATIONS
    parser.add_argument('--nst', type = int, default = nst,
                        help = 'number of thermalisation steps') # number of thermalisation steps
    parser.add_argument('--nsm', type = int, default = nsm,
                        help = 'number of measurements steps') # number of measurement steps
    parser.add_argument('--nips', type = int, default = nips,
                        help = 'number of worm constructions per MC step')
    parser.add_argument('--nrps', type = int, default = nrps,
                        help = 'number of replica loops per MC step')
    parser.add_argument('--measperiod', type = int, default = measperiod,
                        help = 'number of nips worm building + swaps between measurements')
    parser.add_argument('--nb', type = int, default = nb,
                        help = 'number of bins')

    #PARALLELISATION
    parser.add_argument('--ncores', type = int, default = 6,
                        help = 'number of threads to use')

    
    # INITIALISATION PARAMETERS
    parser.add_argument('--randominit', default = True, action ='store_true',
                        help = 'intialise the states randomly')
    parser.add_argument('--same', default = False, action = 'store_true',
                        help = '''initialise all temperatures with the same
                        state (debug purposes)''')
    parser.add_argument('--magninit', default = False, action = 'store_true',
                        help = '''initialise all the temperature with
                        one of the m=1/3 GS''')
    parser.add_argument('--magnstripes', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       m=1/3 stripes''')
    parser.add_argument('--maxflip', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       maximally flippable plateau''')
    parser.add_argument('--loadfromfile', default = False, action = 'store_true',
                       help = '''initialise all the states with
                       results from a previously performed simulations''')
    parser.add_argument('--filename', type = str, default = 'tests/J1hLongRun_RndInit_AlternateUpdateResults',
                       help = '''initialise all the states with
                       results from a previously performed simulations''')
    
    #WORM PARAMETERS
    parser.add_argument('--nmaxiter', type = int, default = 10,
                        help = '''maximal number of segments in a loop update over the
                        size of the lattice (1 = 1times the number of dualbonds in the
                        lattice)''')
    parser.add_argument('--measupdate', default = False, action = 'store_true',
                       help = '''activate to mimic the action of the measuring tip''')
    parser.add_argument('--p', type = float, default = 0.0, 
                       help = '''prob of the measuring tip flipping the spin (number between 0 and 1)''')
    parser.add_argument('--ssf', default = True, action = 'store_true',
                        help = 'activate for single spin flip update')
    parser.add_argument('--alternate', default = True, action = 'store_true',
                        help = 'activate for single spin flip update and dw update')
    parser.add_argument('--checkgs', default =False, action = 'store_true',
                        help = 'activate to debug ssf')
    
    #TEMPERATURE PARAMETERS
    parser.add_argument('--t_list', nargs = '+', type = float, default = [0.05,60],
                        help = 'list of limiting temperature values')
    parser.add_argument('--nt_list', nargs = '+', type = int, default = [nt],
                        help = 'list of number of temperatures in between the given limiting temperatures')
    parser.add_argument('--log_tlist', default = False, action='store_true',
                        help = 'state whether you want the temperature be spaced log-like or linear-like (activate if you want log)')
    parser.add_argument('--stat_temps_lims', nargs = '+', type = float,
                        help = '''limiting temperatures for the various ranges of
                        measurements''') 
                        #default will be set to none, and then we can decide what to do later on.
    #MAGNETIC FIELD PARAMETERS
    parser.add_argument('--h_list', nargs = '+', default = [0.2,6.0],type = float,
                        help = 'list of limiting magnetic field values')
    parser.add_argument('--nh_list', nargs = '+', default = [nh], type = int,
                        help = 'list of number of magnetic fields in between the given limiting temperatures')
    parser.add_argument('--stat_hfields_lims', nargs = '+', type = float,
                help = '''limiting magnetic fields for the various ranges of
                measurements''') 
    #CORRELATIONS PARAMETER
    parser.add_argument('--energy', default = True, action = 'store_true',
                        help = 'activate if you want to save the energy')
    parser.add_argument('--magnetisation', default =True, action = 'store_true',
                        help = 'activate if you want to save the magnetisation')
    parser.add_argument('--charges', default = False, action = 'store_true',
                        help = 'activate if you want to save the charges')
    parser.add_argument('--correlations', default = True, action = 'store_true',
                        help = 'activate if you want to save either central or all correlations')
    parser.add_argument('--all_correlations', default = False, action = 'store_true',
                        help = '''activate if you want to save the correlations for all non-equivalent
                        pairs of sites. Otherwise, will save central correlations.''')
    parser.add_argument('--firstcorrelations', default = False, action = 'store_true',
                        help = 'activate if you want to save first correlations, otherwise will save central')
    #SAVE
    parser.add_argument('--output', type = str, default = "tests", help = 'saving filename (.pkl will be added)')
    args = parser.parse_args()
    return args

In [4]:
def callRunBasis():
    args = genArgs()
    return RunBasis.main(args)

In [5]:
%lprun -f dw.mcs_swaps meanstat, swapst, swapsh, failedupdatesth, failedupdates = callRunBasis() 

-------------------Initialisation--------------------
./tests_folder/backup.hkl
Lattice side size:  12
J1  1
J2  0
J3  0
J3st  0
Couplings extracted
Hamiltonian expression (without field) computed
single spin flip update
alternating ssf and dw update
Number of temperatures:  8
Temperatures: [5.00000000e-02 8.61428571e+00 1.71785714e+01 2.57428571e+01
 3.43071429e+01 4.28714286e+01 5.14357143e+01 6.00000000e+01]
Number of magnetic fields:  8
Magnetic fields:  [0.2        1.02857143 1.85714286 2.68571429 3.51428571 4.34285714
 5.17142857 6.        ]
Fully random initialisation =  True
Identical initialisation =  False
Magnetisation initialisation =  False
List of measurements to be performed: ['Energy', 'Magnetisation', 'Si', 'Central_Correlations']
List of field indices to measure: [0, 1, 2, 3, 4, 5, 6, 7]
-----------Thermalisation------------------
Number of thermalisation steps =  2
itermcs =  2
iterreplicas =  4
ssf =  True
alternate =  True
statsfunctions []
Time for mcsevolve = 0.0

In [6]:
print(failedupdatesth/(nst*nips))

[[0.92322531 0.91338735 0.9211034  0.90528549 0.9222608  0.8755787
  0.88078704 0.87905093]
 [0.16570216 0.18923611 0.19386574 0.24324846 0.28395062 0.34645062
  0.3898534  0.42361111]
 [0.08699846 0.10320216 0.11516204 0.13290895 0.1722608  0.20081019
  0.22492284 0.27044753]
 [0.06462191 0.06674383 0.08217593 0.09915123 0.1242284  0.13117284
  0.16512346 0.19695216]
 [0.04185957 0.05381944 0.05632716 0.07407407 0.09220679 0.1095679
  0.13040123 0.15084877]
 [0.03645833 0.04475309 0.05285494 0.05999228 0.08699846 0.09857253
  0.1095679  0.13136574]
 [0.0310571  0.03665123 0.04301698 0.05324074 0.06076389 0.08024691
  0.09143519 0.10763889]
 [0.02662037 0.03009259 0.0310571  0.04957562 0.05652006 0.07619599
  0.08256173 0.0931713 ]]


In [7]:
print(failedupdates/(nsm*measperiod*nips))

[[1.00012056 1.00034361 0.9991018  1.00036772 1.00034963 1.00028332
  1.00031949 1.00031949]
 [0.16968678 0.19391999 0.19973717 0.24151837 0.28440273 0.32764878
  0.38659216 0.44340157]
 [0.08907817 0.10429929 0.11327522 0.14145086 0.17197748 0.20636212
  0.24045742 0.27702426]
 [0.05949195 0.07051143 0.07854094 0.09943456 0.12237172 0.14528477
  0.17511815 0.20061728]
 [0.0453981  0.05375313 0.06078197 0.07770906 0.09499783 0.1134199
  0.1374361  0.15842617]
 [0.03706115 0.04359568 0.04870153 0.06245177 0.07819131 0.09473259
  0.11122565 0.12843003]
 [0.0300504  0.03669343 0.04038267 0.05249325 0.06594811 0.07996962
  0.09516662 0.11061077]
 [0.02574026 0.03130425 0.03573495 0.04561511 0.05802107 0.06843171
  0.08084973 0.09424431]]


In [8]:
print(4*swapst/(nrps*nsm*measperiod*nh))

[0.         0.0078125  0.22265625 0.44921875 0.64257812 0.7578125
 0.81054688 0.        ]


In [9]:
print(4*swapsh/(nrps*nsm*measperiod*nt))

[0.25       0.47265625 0.43164062 0.41210938 0.4375     0.484375
 0.5390625  0.28515625]
